In [9]:
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from Library.Integration import*
from Library.Root import*
from Library.Diff_eqn import*
import pandas as pd
import prettytable as pt

In [10]:
def poisson_laplace(rho, x_i, x_f, y_i, y_f, u_iy, u_fy, u_xi, u_xf, N, tolerance):
    '''
    Parameters:
    - rho: Function rho(x,y) del^2 u = -rho(x,y)
    - x_i: Initial x value
    - x_f: Final x value
    - y_i: Initial y value
    - y_f: Final y value
    - u_iy: Function u_iy(x)
    - u_fy: Function u_fy(x)
    '''
    '''
    defining the grid N+2 x N+2
    '''
    x = np.linspace(x_i, x_f, N+2)
    y = np.linspace(y_i, y_f, N+2)
    hx = (x_f - x_i) / (N + 1)
    hy = (y_f - y_i) / (N + 1)
    if hx != hy:
        raise ValueError("The grid is not square")
    h = hx 
    A = np.zeros((N*2, N*2))
    '''
    Defining the matrix A
    '''
    for i in range(N**2):
        A[i, i] = 4
        if i == 0:
            A[i, i+1] = -1
            A[i, i+N] = -1
        elif i < N:
            A[i, i-1] = -1
            A[i, i+1] = -1
            A[i, i+N] = -1
        elif i < (N**2 - N):
            A[i, i-1] = -1
            A[i, i+1] = -1
            A[i, i-N] = -1
            A[i, i+N] = -1
        elif i < (N**2 - 1):
            A[i, i-1] = -1
            A[i, i+1] = -1
            A[i, i-N] = -1
        else:
            A[i, i-1] = -1
            A[i, i-N] = -1
    '''
    Defining the matrix B
    '''
    B = []
    for i in range(1,N+1):
        for j in range(1,N+1):
            sum = rho(x[i], y[j]) * h**2
            if i == 0:
                sum += u_xi(y[j])
            if i == N:
                sum += u_xf(y[j])
            if j == 0:
                sum += u_iy(x[i])
            if j == N:
                sum += u_fy(x[i])
            B.append(sum)    
    B = np.array(B)[:,None]
    # solving the system of linear equations
    # u = lib.solve_GS(A, B, tolerance)
    u = np.linalg.solve(A, B)
    u = np.array(u).reshape((N,N))
    u = np.append(u_iy(y[1:-1,None]), u, axis = 1)
    u = np.append(u, u_fy(y[1:-1, None]), axis = 1)
    u = np.append([u_xi(x)], u, axis = 0)
    u = np.append(u, [u_xf(x)], axis = 0)
    
    return x, y, u

In [11]:
def rho(x, y): 
    return x * np.exp(y)
def u_iy(y):
    return np.zeros(y.shape)
def u_fy(y):
    return 2 * np.exp(y)
def u_xi(x):
    return x
def u_xf(x):
    return x * np.e
'''
Defining the limits and Number of subintervals and other parameters as per the Question
'''
grid_size = 6
x_i = 0
x_f = 2
y_i = 0
y_f = 2
tolerance = 1e-6

x, y, u = poisson_laplace(rho, x_i, x_f, y_i, y_f, u_iy, u_fy, u_xi, u_xf, grid_size, tolerance=tolerance)

IndexError: index 12 is out of bounds for axis 1 with size 12

In [ ]:
print(f"OUTPUT\n------")
print(f"The solution of the given Poisson , u(x,y), found with grid size {grid_size} is as follows:")
table4 = pt.PrettyTable()
table4.field_names = ["y"] + [f"{x[i]:.2f}" for i in range(len(x))]
for i in range(len(y)):
    row = [f"{y[i]:.2f}"]
    for j in range(len(x)):
        row.append(f"{u[i,j]:.6f}")
    table4.add_row(row)
print(table4)

# plotting the solution

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(x, y)
surf = ax.plot_surface(X, Y, u, cmap='viridis')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('u(x, y)')
ax.set_title("Solution of the given Poisson equation")
fig.colorbar(surf)
plt.show()

OUTPUT
------
The solution of the given Poisson , u(x,y), found with grid size 6 is as follows:


NameError: name 'x' is not defined